<a href="https://colab.research.google.com/github/EasonHu0620/huashan_exh/blob/main/huashan_exh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
import re
import csv
import time
from urllib.parse import urljoin  #網址合併用

import requests as req
from bs4 import BeautifulSoup as bs
from requests.utils import requote_uri

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

base_url = "https://www.huashan1914.com"
exh = "https://www.huashan1914.com/w/huashan1914"
museum_name = "華山1914文創園區"
row = []

def get_driver(headless=True):
    from selenium.webdriver.chrome.options import Options
    opts = Options()
    if headless:
      opts.add_argument("--headless=new") #讓瀏覽器在背景執行
    opts.add_argument("--window-size=max") #設定瀏覽器啟動時的視窗大小
    opts.add_argument("--lang=zh-TW") #設定瀏覽器的語言首選為繁體中文
    opts.add_argument("--disable-gpu") #headless 模式穩定
    opts.add_argument("--no-sandbox") #避免在某些環境無法啟動的錯誤
    return webdriver.Chrome(options=opts)

driver = get_driver(headless=True)
rows = []
try:
  driver.get(exh)
  wait = WebDriverWait(driver, 20)

  # 等待我要的class出現
  container = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".swiper-slide.swiper-slide-active")))
  items = container.find_elements(By.XPATH, "./div")
  for it in items:
    #展覽連結
    ex_link = ""
    try:
      img = it.find_element(By.XPATH, './img')
      link = img.get_attribute("onclick")
      url_part = re.search(r"'(/[^']+)'", link).group(1)
      ex_link = urljoin(base_url, url_part)
    except Exception:
      pass

    if not ex_link.startswith(("http://", "https://")):
      continue
    resp = req.get(ex_link)
    html = bs(resp.text, "html.parser")

     #展覽名稱
    title = ""
    try:
      ex_title = html.find("div", class_="article-title page")
      title = ex_title.text.strip()
    except Exception:
      pass


    #展覽日期
    ex_date = ""
    try:
      dates = [d.get_text(strip=True) for d in html.find_all("div", class_="card-date")]
      ex_date = " - ".join(dates[:2]) if dates else ""
    except Exception:
      pass

    #展覽時間
    ex_time = ""
    try:
      node = html.find("div", class_="card-time")
      if node:
        raw = node.get_text(" ", strip=True)
        # 如果不是數字開頭就略過
        if re.match(r'^\d', raw):
          ex_time = raw
        else:
          ex_time = ""
    except Exception:
        pass

    #展覽圖片
    ex_img = ""
    try:
      first_img = html.select_one("span[rel] img")
      if first_img:
        ex_img = requote_uri(urljoin(base_url, first_img.get("src")))
    except Exception:
        pass

    #展覽地點
    ex_place = ""
    try:
      ex_place = html.find("a", class_="openMap").get_text(strip=True)
    except Exception:
      pass

    rows.append({
      "館別": museum_name,
      "展覽名稱": title,
      "展覽地點": ex_place,
      "展覽日期": ex_date,
      "展覽時間": ex_time,
      "展覽連結": ex_link,
      "展覽圖片": ex_img
    })
finally:
  driver.quit()
# ★ 寫出 CSV 檔（UTF-8 with BOM，Excel 也能直接開）
fieldnames = ["館別", "展覽名稱", "展覽地點", "展覽日期", "展覽時間", "展覽連結", "展覽圖片"]
with open("huashan_exhibitions.csv", "w", newline="", encoding="utf-8-sig") as f:
  writer = csv.DictWriter(f, fieldnames=fieldnames)
  writer.writeheader()
  writer.writerows(rows)

print(f"已輸出 {len(rows)} 筆到 huashan_exhibitions.csv")





已輸出 5 筆到 huashan_exhibitions.csv
